In [1]:
import tkinter as tk
import numpy as np
import time
from tkinter import Canvas, Tk, mainloop, Toplevel
from tkinter import filedialog
from tkinter import messagebox
from PIL import Image, ImageTk, ImageDraw
from tkinter.font import Font
from cv2 import createCLAHE
from keras.models import load_model
from scipy import misc
from skimage.filters import unsharp_mask
from skimage import segmentation
from skimage import color
import matplotlib.pyplot as plt
import imageio

cnn = load_model('cnn_NP_vs_P.h5', compile = False)

feature_name = ['NP', 'P'] 
clahe= createCLAHE(clipLimit=3.0)

In [2]:
def Crop_img():
    
    root = Toplevel()
    root.title('Crop Image')
    crop_img = Image.open(imgpath_Var.get())
    width, height = crop_img.size

    ca = Canvas(root, width=width, height=height)
    ca.pack()
    photoimg = ImageTk.PhotoImage("RGB", crop_img.size)
    photoimg.paste(crop_img)
    ca.create_image(width//2,height//2, image=photoimg)

    loc = []
    images = []
    temp_var = tk.IntVar()
    temp_var.set(0)

    def create_rectangle(x1, y1, x2, y2, **kwargs):
        if 'alpha' in kwargs:
            alpha = int(kwargs.pop('alpha') * 255)
            fill = kwargs.pop('fill')
            fill = root.winfo_rgb(fill) + (alpha,)
            image = Image.new('RGBA', (x2-x1, y2-y1), fill)
            images.append(ImageTk.PhotoImage(image))
            ca.create_image(x1, y1, image=images[-1], anchor='nw')
        ca.create_rectangle(x1, y1, x2, y2, **kwargs)

    def callback1(event):
        
        if len(loc)>=1:
            loc.clear()
        
        print('左鍵當前位置為：',event.x, event.y)
        x1,y1 = (event.x-2),(event.y-2)
        x2,y2 = (event.x+2),(event.y+2)
        create_rectangle(x1,y1,x2,y2,fill="red")
        loc.append((event.x, event.y))

    def callback2(event):
        print('右鍵當前位置為：',event.x, event.y)
        x1,y1 = (event.x-2),(event.y-2)
        x2,y2 = (event.x+2),(event.y+2)
        create_rectangle(x1,y1,x2,y2,fill="red")
        create_rectangle(loc[0][0],loc[0][1], event.x, event.y, fill="yellow", alpha=.5)

        img_copy = crop_img.copy()
        img_crop = img_copy.crop((loc[0][0], loc[0][1], event.x, event.y))
        time_str = time.strftime("%Y%m%d_%H%M%S", time.localtime()) #現在時間格式化成20160320 114539形式
        img_crop.save("Teeth"+ time_str +".jpg")
        
        img_crop_resize = img_crop.resize((40,80))
        imgcroptk = ImageTk.PhotoImage(img_crop_resize)
        
        temp_number = temp_var.get()
        
        label_img_list[temp_number].config(image = imgcroptk)
        label_img_list[temp_number].image = imgcroptk
        
        temp_var.set(temp_number+2)
        imgpath_crop_Var_list[int(temp_number/2)].set("Teeth"+ time_str +".jpg")
        
        loc.clear()

    ca.bind('<Button-1>',callback1)
    ca.bind('<Button-3>',callback2)
    ca.pack()
    mainloop()

In [3]:
#建立選擇欄
def NewFile():
    print("New File!")
def About():
    messagebox.showinfo(title='About', message="本研究東華大學測試使用!")

In [4]:
def Open_img():
    
    filename = filedialog.askopenfilename(title='Select file', filetypes=[("all files","*.*"),("bmp files","*.bmp"),("png files","*.png"),("jpeg files","*.jpg")])
    
    img = Image.open(filename).resize((300,200))
    imgtk = ImageTk.PhotoImage(img)
    label_img.config(image = imgtk)
    label_img.image = imgtk
    imgpath_Var.set(filename)

In [5]:
def Predict_img():
    
    if imgpath_crop_Var_list[0].get() != "":
        img = Image.open(imgpath_crop_Var_list[0].get()).convert('L')
        img_np = np.array(img, dtype="uint8")
        img_clahe = clahe.apply(img_np)
        img_unsharp = unsharp_mask(img_clahe, radius=3, amount=2)
        img_seg = segmentation.slic(img_unsharp, n_segments=4000, compactness=0.1, start_label=0)
        img_label2 = color.label2rgb(img_seg, img_unsharp, kind='avg')
        imageio.imwrite('C_U_S0.bmp', img_label2)
        img = Image.open("C_U_S0.bmp").resize((100,100))
        img_np = np.asarray(img, dtype="uint8")
        img_nor = img_np.astype('float32')/255
        predict = cnn.predict(np.array([img_nor.reshape((100,100,1))]))
        predictions = [np.argmax(one_hot)for one_hot in predict]
        label_predict_list[0].config(text = feature_name[predictions[0]])
        
        img_config = img.resize((40,80))
        imgcroptk = ImageTk.PhotoImage(img_config)
        label_img_list[1].config(image = imgcroptk)
        label_img_list[1].image = imgcroptk
    
    if imgpath_crop_Var_list[1].get() != "":
        img = Image.open(imgpath_crop_Var_list[1].get()).convert('L')
        img_np = np.array(img, dtype="uint8")
        img_clahe = clahe.apply(img_np)
        img_unsharp = unsharp_mask(img_clahe, radius=3, amount=2)
        img_seg = segmentation.slic(img_unsharp, n_segments=4000, compactness=0.1, start_label=0)
        img_label2 = color.label2rgb(img_seg, img_unsharp, kind='avg')
        imageio.imwrite('C_U_S1.bmp', img_label2)
        img = Image.open("C_U_S1.bmp").resize((100,100))
        img_np = np.asarray(img, dtype="uint8")
        img_nor = img_np.astype('float32')/255
        predict = cnn.predict(np.array([img_nor.reshape((100,100,1))]))
        predictions = [np.argmax(one_hot)for one_hot in predict]
        label_predict_list[1].config(text = feature_name[predictions[0]])
        
        img_config = img.resize((40,80))
        imgcroptk = ImageTk.PhotoImage(img_config)
        label_img_list[3].config(image = imgcroptk)
        label_img_list[3].image = imgcroptk
        
    if imgpath_crop_Var_list[2].get() != "":
        img = Image.open(imgpath_crop_Var_list[2].get()).convert('L')
        img_np = np.array(img, dtype="uint8")
        img_clahe = clahe.apply(img_np)
        img_unsharp = unsharp_mask(img_clahe, radius=3, amount=2)
        img_seg = segmentation.slic(img_unsharp, n_segments=4000, compactness=0.1, start_label=0)
        img_label2 = color.label2rgb(img_seg, img_unsharp, kind='avg')
        imageio.imwrite('C_U_S2.bmp', img_label2)
        img = Image.open("C_U_S2.bmp").resize((100,100))
        img_np = np.asarray(img, dtype="uint8")
        img_nor = img_np.astype('float32')/255
        predict = cnn.predict(np.array([img_nor.reshape((100,100,1))]))
        predictions = [np.argmax(one_hot)for one_hot in predict]
        label_predict_list[2].config(text = feature_name[predictions[0]])
        
        img_config = img.resize((40,80))
        imgcroptk = ImageTk.PhotoImage(img_config)
        label_img_list[5].config(image = imgcroptk)
        label_img_list[5].image = imgcroptk
    

In [6]:
app = tk.Tk()
app.title('Teeth AI')
app.geometry('500x480')

#建立菜單
menu = tk.Menu(app)
app.config(menu=menu)

#建立菜單的file
file_menu = tk.Menu(menu, tearoff = 0) #是否有虛線，無虛線則無法分離視窗
menu.add_cascade(label="File", menu=file_menu)

#file建立新增
file_menu.add_command(label="New", command=NewFile)

#file建立虛線
file_menu.add_separator()

#file建立離開
file_menu.add_command(label="Exit", command=app.destroy)

#建立菜單的幫助
help_menu = tk.Menu(menu, tearoff = 0)
menu.add_cascade(label="Help", menu=help_menu)
help_menu.add_command(label="About", command=About)

#設立字體
subtitle_label_Font_1 = Font(family="Times", size=12)

#建立畫布與位置
frame_title = tk.Frame(app)
frame_title.grid(column=0, row=0, ipadx=5, pady=5)

frame_img = tk.LabelFrame(app, text='Image show')  
frame_img.grid(column=0, row=1, ipadx=0, padx=10, pady=10, sticky=tk.W+tk.N)  

frame_but = tk.LabelFrame(app, text='Open/Crop') 
frame_but.grid(column=1, row=1, ipadx=0, padx=10, pady=10, sticky=tk.W+tk.N)  

frame_output = tk.LabelFrame(app, text='Crop Image')
frame_output.grid(column=0, row=2, ipadx=0, padx=10, pady=10, sticky=tk.W+tk.N)

# 建立文字
label_Subtitle = tk.Label(frame_img, text="Please enter the image.")
label_Subtitle.grid(column=0, row=0, ipadx=0, pady=0)

#GUI影像
img_path = "p-img.png"
img = Image.open(img_path).resize((300,200))
imgtk = ImageTk.PhotoImage(img)
label_img = tk.Label(frame_img, image=imgtk)
label_img.grid(column=0, row=1, ipadx=10, pady=10)

# 影像放置位置
img_path_list = ["p-img.png","p-img.png","p-img.png","p-img.png","p-img.png","p-img.png"]
imgtk_list, label_img_list = [], []

for i in range(len(img_path_list)):
    simg = Image.open(img_path_list[i]).resize((40,80))
    simgtk = ImageTk.PhotoImage(simg)
    imgtk_list.append(simgtk)
    slabel_img = tk.Label(frame_output, image=imgtk_list[i])
    label_img_list.append(slabel_img)
    label_img_list[i].grid(column=i, row=0, ipadx=5, pady=5)

#影像位置變數
imgpath_Var = tk.StringVar()
imgpath_crop_Var_list = []
for i in range(3):
    imgpath_crop_Var_list.append(tk.StringVar())

#插入影像按鈕
imgButton = tk.Button(frame_but, text = 'Open Image',command = Open_img)
imgButton.grid(column=0, row=1, padx=10, pady=10)

#裁剪按鈕
cropButton = tk.Button(frame_but, text = 'Crop Image',command = Crop_img)
cropButton.grid(column=0, row=2, padx=10, pady=10)

predictButton = tk.Button(frame_but, text = 'Predict Image',command = Predict_img)
predictButton.grid(column=0, row=3, padx=10, pady=10)

# 建立文字
label_predict_list = []

for i in range(int(len(img_path_list)/2)):
    label_predict = tk.Label(frame_output, text="None")
    label_predict_list.append(label_predict)
    label_predict_list[i].grid(column=i+i, row=1, ipadx=0, pady=0)
    
mainloop()

左鍵當前位置為： 128 57
右鍵當前位置為： 309 338
右鍵當前位置為： 311 39


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\envs\py36\lib\tkinter\__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-2-5fcdf2a6bfc9>", line 45, in callback2
    create_rectangle(loc[0][0],loc[0][1], event.x, event.y, fill="yellow", alpha=.5)
IndexError: list index out of range


左鍵當前位置為： 312 40
右鍵當前位置為： 500 355
左鍵當前位置為： 494 52
右鍵當前位置為： 617 377


C:\ProgramData\Anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: The new recommended value for bg_label is 0. Until version 0.19, the default bg_label value is -1. From version 0.19, the bg_label default value will be 0. To avoid this warning, please explicitly set bg_label value.
  if __name__ == '__main__':
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
C:\ProgramData\Anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:29: FutureWarning: The new recommended value for bg_label is 0. Until version 0.19, the default bg_label value is -1. From version 0.19, the bg_label default value will be 0. To avoid this warning, please explicitly set bg_label value.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
C:\ProgramData\Anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:49: FutureWarning: The new recommende

左鍵當前位置為： 119 49
右鍵當前位置為： 325 333
左鍵當前位置為： 304 37
右鍵當前位置為： 509 345
左鍵當前位置為： 486 50
右鍵當前位置為： 621 333


C:\ProgramData\Anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: The new recommended value for bg_label is 0. Until version 0.19, the default bg_label value is -1. From version 0.19, the bg_label default value will be 0. To avoid this warning, please explicitly set bg_label value.
  if __name__ == '__main__':
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


左鍵當前位置為： 26 88
右鍵當前位置為： 171 395
左鍵當前位置為： 147 96
右鍵當前位置為： 353 407
左鍵當前位置為： 335 105
右鍵當前位置為： 542 417


Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
